<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MC_Antit%C3%A9ticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import time
from random import random

In [83]:
def g(x):
  return np.sqrt(np.arctan(x))

In [84]:
n = 5000
repeticiones=50

##**Método Crudo**

In [85]:
def crudo_antitetico(n):
    suma_r = 0
    suma_1r = 0
    for i in range(n):
        r = rd.random()
        suma_r += g(r)          # theta1(r)
        suma_1r += g(1-r)       # theta1(1-r)
    # Promedios individuales y combinado
    I1 = suma_r / n
    I2 = suma_1r / n
    I12 = (I1 + I2) / 2
    return I1, I2, I12

In [86]:
def evaluar_antitetico(n, repeticiones):
    inicio_tiempo = time.time()

    lista_I1 = []
    lista_I2 = []
    lista_I12 = []

    for _ in range(repeticiones):
        i1, i2, i12 = crudo_antitetico(n)
        lista_I1.append(i1)
        lista_I2.append(i2)
        lista_I12.append(i12)

    fin_tiempo = time.time()
    tiempo_total = fin_tiempo - inicio_tiempo

    # Medias y varianzas de cada caso
    mu_I1, var_I1 = np.mean(lista_I1), np.var(lista_I1)
    mu_I2, var_I2 = np.mean(lista_I2), np.var(lista_I2)
    mu_I12, var_I12 = np.mean(lista_I12), np.var(lista_I12)

    return tiempo_total, (mu_I1, var_I1), (mu_I2, var_I2), (mu_I12, var_I12)

In [87]:
tiempo, I1, I2, I12 = evaluar_antitetico(n, repeticiones)

In [88]:
print("   Estimador        Media       Varianza")
print("-----------------------------------------------")
print(f"  theta1(r)     {I1[0]:.6f}   {I1[1]:.6f}")
print(f"  theta1(1-r)   {I2[0]:.6f}   {I2[1]:.6f}")
print(f"  Antitético    {I12[0]:.6f}   {I12[1]:.6f}")
print("-----------------------------------------------")
print(f"Tiempo total: {tiempo:.4f}s")

   Estimador        Media       Varianza
-----------------------------------------------
  theta1(r)     0.630263   0.000006
  theta1(1-r)   0.629280   0.000006
  Antitético    0.629771   0.000000
-----------------------------------------------
Tiempo total: 1.0967s


##**Método Acierto y error**

In [89]:
def aciertoerror_antitetico(n):
    aciertos_r = 0
    aciertos_1r = 0
    for i in range(n):
        x = rd.random()
        y = rd.random()
        # Usamos r
        if y <= g(x):
            aciertos_r += 1
        # Usamos 1-r
        if y <= g(1-x):
            aciertos_1r += 1
    I1 = aciertos_r / n
    I2 = aciertos_1r / n
    I12 = (I1 + I2) / 2
    return I1, I2, I12

In [90]:
def evaluar_aciertoerror_antitetico(n, repeticiones):
    inicio_tiempo = time.time()

    lista_I1 = []
    lista_I2 = []
    lista_I12 = []

    for _ in range(repeticiones):
        i1, i2, i12 = aciertoerror_antitetico(n)
        lista_I1.append(i1)
        lista_I2.append(i2)
        lista_I12.append(i12)

    fin_tiempo = time.time()
    tiempo_total = fin_tiempo - inicio_tiempo

    mu_I1, var_I1 = np.mean(lista_I1), np.var(lista_I1)
    mu_I2, var_I2 = np.mean(lista_I2), np.var(lista_I2)
    mu_I12, var_I12 = np.mean(lista_I12), np.var(lista_I12)

    return tiempo_total, (mu_I1, var_I1), (mu_I2, var_I2), (mu_I12, var_I12)

In [91]:
tiempo, I1, I2, I12 = evaluar_aciertoerror_antitetico(n, repeticiones)

In [92]:
print("   Estimador        Media       Varianza")
print("-----------------------------------------------")
print(f"  theta1(r)     {I1[0]:.6f}   {I1[1]:.6f}")
print(f"  theta1(1-r)   {I2[0]:.6f}   {I2[1]:.6f}")
print(f"  Antitético    {I12[0]:.6f}   {I12[1]:.6f}")
print("-----------------------------------------------")
print(f"Tiempo total: {tiempo:.4f}s")

   Estimador        Media       Varianza
-----------------------------------------------
  theta1(r)     0.629908   0.000057
  theta1(1-r)   0.626948   0.000053
  Antitético    0.628428   0.000036
-----------------------------------------------
Tiempo total: 1.0789s
